# `ARGON RIADI`
# `HCK 004`

#Objective
Pada laporan ini saya akan mengolah dataset dari sebuah Ecommerce bernama 'The Look' untuk melihat performance bisnis mereka selama tahun 2022

#OUTPUT
Menemukan masalah perihal metriks mana yang harus diperbaiki demi meningkatkan pendapatan perusahaan

# Import Data Library

In [138]:
# Melakukan import dataset thelook_ecommerce dari Google bigquery ke Google Collab

from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')

project_id = "p0-gc3-378001" #GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
client = bigquery.Client(project=project_id)

Authenticated


In [139]:
# Melakukan loading dataset thelook_ecommerce Google Collab

df = client.query('''
select *
from `bigquery-public-data.thelook_ecommerce.orders`''').to_dataframe()
df

,order_id,user_id,status,gender,created_at,returned_at,shipped_at,delivered_at,num_of_item
0,30,24,Shipped,F,2019-10-08 01:04:00+00:00,NaT,2019-10-10 13:54:00+00:00,NaT,4
1,36,29,Shipped,F,2020-07-11 04:15:00+00:00,NaT,2020-07-12 09:38:00+00:00,NaT,1
2,45,38,Shipped,F,2020-04-15 04:37:00+00:00,NaT,2020-04-16 08:04:00+00:00,NaT,1
3,56,48,Shipped,F,2022-06-29 09:10:00+00:00,NaT,2022-07-01 23:42:00+00:00,NaT,1
4,66,56,Shipped,F,2022-07-04 17:27:00+00:00,NaT,2022-07-05 08:15:00+00:00,NaT,2
...,...,...,...,...,...,...,...,...,...
124643,124623,99976,Processing,M,2021-03-16 11:56:00+00:00,NaT,NaT,NaT,2
124644,124624,99977,Processing,M,2022-08-06 03:41:00+00:00,NaT,NaT,NaT,1
124645,124627,99980,Processing,M,2022-05-08 08:03:00+00:00,NaT,NaT,NaT,2
124646,124628,99981,Processing,M,2020-10-28 10:12:00+00:00,NaT,NaT,NaT,1


# Penjabaran 1 - Jumlah Transaksi

Melihat jumlah transaksi selama Q1 sampai Q3 di tahun 2022
* hanya yang transaksi berstatus 'Complete'

In [140]:
#Melakukan query dengan fungsi groupby untuk menghitung jumlah transaksi menggunakan command 'count' di kolom order id
#table yang digunakan: order items

df2 = client.query('''
SELECT extract(year from created_at) as year, extract(month from created_at) as month, count(order_id) as jumlah_transaksi
FROM `bigquery-public-data.thelook_ecommerce.order_items` 
WHERE status='Complete' and created_at between "2022-01-01"and "2022-9-30 23:59:00+00:00"
GROUP BY year,month
ORDER BY year,month ASC
''').to_dataframe()
df2

,year,month,jumlah_transaksi
0,2022,1,1246
1,2022,2,1119
2,2022,3,1485
3,2022,4,1308
4,2022,5,1548
5,2022,6,1531
6,2022,7,1780
7,2022,8,1870
8,2022,9,1965


Insight Penjabaran 1

- Dari data di atas dapat terlihat bahwa secara keseluruhan terjadi peningkatan jumlah transaksi selama periode 2022 dari Q1 hingga Q2

- Jumlah transaksi tertinggi terjadi di akhir Q3 (Sep 2022)

# Penjabaran 2 - Total Penjualan

Melihat total penjualan (dalam USD) selama Q1 sampai Q3 di tahun 2022 

* hanya yang transaksi berstatus 'Complete'

In [141]:
#Melakukan query dengan fungsi groupby untuk menghitung total penjualan (dalam USD) menggunakan command 'sum' di kolom sale price
#table yang digunakan: inventory items & order items

df3 = client.query('''
SELECT extract(year from items.created_at) as year, extract(month from items.created_at) as month, round(sum(items.sale_price),0) as total_penjualan
FROM `bigquery-public-data.thelook_ecommerce.inventory_items` as inventory
join  `bigquery-public-data.thelook_ecommerce.order_items` as items
on items.inventory_item_id = inventory.id
WHERE items.status='Complete' and items.created_at between "2022-01-01"and "2022-9-30 23:59:00+00:00"
GROUP BY year, month
ORDER BY year, month ASC
''').to_dataframe()
df3

,year,month,total_penjualan
0,2022,1,73116.0
1,2022,2,60195.0
2,2022,3,87891.0
3,2022,4,73997.0
4,2022,5,94903.0
5,2022,6,88125.0
6,2022,7,107516.0
7,2022,8,107630.0
8,2022,9,116001.0


Insight

- Dari data di atas dapat terlihat bahwa secara keseluruhan terjadi peningkatan total penjualan selama periode 2022 dari Q1 hingga Q3

- Total penjualan tertinggi terjadi di akhir Q3 (Sep 2022)

# Penjabaran 3 - Jumlah User

Melihat jumlah user yang melakukan transaksi dari Q1 sampai Q3 2022
* hanya yang transaksi berstatus 'Complete'

In [142]:
#Melakukan query dengan fungsi groupby untuk menghitung total user menggunakan command 'count(distinct)' di kolom user id
#table yang digunakan: order items

df4 = client.query('''
SELECT extract(year from items.created_at) as year, extract(month from items.created_at) as month, count(distinct items.user_id) as total_user
FROM `bigquery-public-data.thelook_ecommerce.order_items` as items
WHERE items.status='Complete' and items.created_at between "2022-01-01"and "2022-9-30 23:59:00+00:00"
GROUP BY year, month
ORDER BY year, month ASC
''').to_dataframe()
df4

,year,month,total_user
0,2022,1,850
1,2022,2,786
2,2022,3,1009
3,2022,4,924
4,2022,5,1066
5,2022,6,1072
6,2022,7,1248
7,2022,8,1285
8,2022,9,1370


Insight

- Dari data di atas dapat terlihat bahwa secara keseluruhan terjadi peningkatan jumlah user yang melakukan transaksi selama periode 2022 dari Q1 hingga Q2

- Jumlah user yang melakukan transaksi tertinggi terjadi di akhir Q3 (Sep 2022)

# Penjabaran 4 - Kategori Produk

Melihat kategori produk yang paling banyak dibeli dari Q1 sampai Q3 2022
* hanya yang transaksi berstatus 'Complete'

In [143]:
#Melakukan subquery dengan fungsi groupby untuk menghitung kategori produk menggunakan command 'row_number()' & 'count' di kolom product category
#table yang digunakan: inventory items & order items

df5 = client.query('''
select * 
from
(select *, row_number() over (partition by month order by produk_terjual desc) as ranking
from
(select year, month, product_category, count(product_category) as produk_terjual
from
(select*, extract(year from items.created_at) as year, extract(month from items.created_at) as month
from bigquery-public-data.thelook_ecommerce.inventory_items as items
join bigquery-public-data.thelook_ecommerce.order_items as orders
on items.id = orders.id)
where status = "Complete" and month <10 and year = 2022
group by year, month, product_category)
group by year, month, product_category, produk_terjual)
where ranking = 1
order by month
''').to_dataframe()
df5

,year,month,product_category,produk_terjual,ranking
0,2022,1,Sleep & Lounge,101,1
1,2022,2,Jeans,90,1
2,2022,3,Intimates,109,1
3,2022,4,Intimates,97,1
4,2022,5,Fashion Hoodies & Sweatshirts,106,1
5,2022,6,Intimates,113,1
6,2022,7,Shorts,111,1
7,2022,8,Intimates,116,1
8,2022,9,Sleep & Lounge,120,1


Insight
- kategori 'Intimates' menjadi kategori dengan produk penjualan tertinggi dibandingkan kategori lainnya dalam periode Q12 sampai Q3 2022

# Penjabaran 5 - Kategori Produk (Pendapatan)

Melihat kategori produk yang paling banyak mendapatkan pendapatan dari Q1 sampai Q3 2022
* hanya yang transaksi berstatus 'Complete'

In [144]:
#Melakukan subquery dengan fungsi groupby untuk menghitung kategori produk menggunakan command 'row_number()' & 'count' di kolom product category
#table yang digunakan: inventory items & order items

df6 = client.query('''
select *
from
(select *, dense_rank() over (partition by month order by pendapatan desc) as ranking
from
(select year, month, product_category, round(sum(sale_price),0) as pendapatan
from
(select  *, extract(year from items.created_at) as year, extract(month from items.created_at) as month
from bigquery-public-data.thelook_ecommerce.inventory_items as items
join bigquery-public-data.thelook_ecommerce.order_items as orders
on items.id = orders.inventory_item_id)
where status = "Complete" and month <10 and year = 2022
group by year, month, product_category)
group by year, month, product_category, pendapatan
order by year, month)
where ranking = 1
order by month
''').to_dataframe()
df6

,year,month,product_category,pendapatan,ranking
0,2022,1,Outerwear & Coats,11727.0,1
1,2022,2,Outerwear & Coats,7905.0,1
2,2022,3,Jeans,8080.0,1
3,2022,4,Outerwear & Coats,12769.0,1
4,2022,5,Outerwear & Coats,10153.0,1
5,2022,6,Jeans,8937.0,1
6,2022,7,Outerwear & Coats,8181.0,1
7,2022,8,Outerwear & Coats,9758.0,1
8,2022,9,Outerwear & Coats,8487.0,1


Insight
- kategori 'Outwear & Coats' menjadi kategori dengan produk penjualan tertinggi dibandingkan kategori lainnya dalam periode Q1 sampai Q3 2022

# Penjabaran 6 - Perbandingan Total Penjualan & Jumlah Transaksi dengan Jumlah User 

Membandingkan antara jumlah transaksi dan total penjualan, mana yang paling berkaitan dengan jumlah user yang melakukan transaksi

In [145]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import pearsonr
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt

In [146]:
#mencari korelasi pearson antara total penjualan dengan jumlah user yang melakukan transaksi

r_penjualan_user = stats.pearsonr(df3["total_penjualan"], df4["total_user"])[0]
r_penjualan_user

0.9846868204451702

In [147]:
#mencari korelasi pearson antara jumlah transaksi dengan jumlah user yang melakukan transaksi

r_transaksi_user = stats.pearsonr(df2["jumlah_transaksi"],df4["total_user"])[0]
r_transaksi_user

0.9975808292307065

Insight

- Dari dua data di atas, terlihat bahwa angka total penjualan & angka jumlah transaksi memiliki korelasi erat dengan angka jumlah user yang melakukan transaksi

- Korelasi jumlah transaksi menjadi perbandingan yang paling berkaitan dengan jumlah user yang melakukan transaksi

# Pertanyaan 1 - Mengukur tingkat pencapaian problem statement dengan metode SMART

Mengukur tingkat pencapaian problem statement yang sudah saya utarakan di awal laporan: 

"Metriks mana yang harus diperbaiki demi meningkatkan pendapatan perusahaan?"

- Specific:
Upaya meningkatkan pendapatan perusahaan

- Measurable:
Meningkatkan total pendapatan perusahaan sebesar 32%

- Attainable:
Dari perhitungan korelasi sebelumnya, terlihat bahwa apabila kita mampu meningkatkan jumlah transaksi tiap bulan maka akan terjadi pula tingkat pendapatan perusahaan

- Relevant:
Dengan berfokus pada upaya upselling kepada para user, kita harus mencoba untuk melakukan aktivitas campaign yang bertujukan untuk meningkatkan angka transaksi seperti: melakukan promo potongan harga dengan minimum transaksi, meningkatkan push notifikasi ketersediaan barang yang dibutuhkan sesuai minat user

- Time Bound:
Akan terlihat efektivitas angka peningkatan transaksi saat periode campaign berakhir (end of week / end of month)

# Pertanyaan 2 - Pendapatan USD250,000 di awal kuartal 4 

Berdasarkan hasil analisis yang sudah kamu lakukan dari 6 penjabaran di atas, jika CEO perusahaanmu ingin menargetkan pendapatan di awal kuartal 4 harus mencapai USD250,000, apakah masuk akal?

- Jawaban:
Menjadi target yang berat karena range rerata kenaikan pendapatan The Look tiap bulannya adalah 7% - 46% 

*(range target optimis pendapatan di awal kuartal 4 adalah USD120,000 - USD169,000) 

# Pertanyaan 3 - Mendapatkan investor baru

CEO menargetkan di kuartal 4 ada investor yang dapat menyuntikan dana ke perusahaan, 

Informasi yang dibutuhkan oleh CEO:
- Trend transaksi
- Trend jumlah user
- GMV selama 3 kuartal


In [148]:
# Data Transaksi Q1 - Q3 2022

df2

,year,month,jumlah_transaksi
0,2022,1,1246
1,2022,2,1119
2,2022,3,1485
3,2022,4,1308
4,2022,5,1548
5,2022,6,1531
6,2022,7,1780
7,2022,8,1870
8,2022,9,1965


Insight

Dari data transaksi di atas terlihat bahwa secara keseluruhan terjadi trend kenaikan jumlah transaksi selama periode Q1 Q3 2022 dengan rerata range kenaikan sebesar 5% - 32%, namun memang angka kenaikan di tiap bulan masih belum stabil 

In [149]:
# Data jumlah user yang melakukan transaksi Q1 - Q3 2022

df4

,year,month,total_user
0,2022,1,850
1,2022,2,786
2,2022,3,1009
3,2022,4,924
4,2022,5,1066
5,2022,6,1072
6,2022,7,1248
7,2022,8,1285
8,2022,9,1370


Insight

Dari data transaksi di atas terlihat bahwa secara keseluruhan terjadi trend kenaikan total user yang melakukan transaksi selama periode Q1 Q3 2022 dengan rerata kenaikan total user dari range 0.5% - 15%, trend kenaikan dari sisi user ini masih lebih menjanjikan dika dibandingkan trend kenaikan jumlah transaksi

In [150]:
#Menyiapkan dataset untuk menghitung GMV selama 3 kuartal terakhir

df10 = client.query('''
select *,extract(year from orders.created_at) as year, extract(month from orders.created_at) as month, extract(quarter from orders.created_at) as quarter, extract(week from orders.created_at) as week
from `bigquery-public-data.thelook_ecommerce.orders` as orders
join `bigquery-public-data.thelook_ecommerce.order_items` as items
on orders.order_id = items.order_id
''').to_dataframe()
df10

,order_id,user_id,status,gender,created_at,returned_at,shipped_at,delivered_at,num_of_item,id,...,status_1,created_at_1,shipped_at_1,delivered_at_1,returned_at_1,sale_price,year,month,quarter,week
0,84220,67533,Complete,F,2022-01-01 13:41:00+00:00,NaT,2022-01-04 10:56:00+00:00,2022-01-08 09:09:00+00:00,1,122658,...,Complete,2022-01-01 12:40:00+00:00,2022-01-04 10:56:00+00:00,2022-01-08 09:09:00+00:00,NaT,6.000000,2022,1,1,0
1,120689,96777,Returned,F,2022-01-01 07:04:00+00:00,2022-01-05 18:10:00+00:00,2022-01-03 04:22:00+00:00,2022-01-03 18:48:00+00:00,1,175242,...,Returned,2022-01-01 03:15:35+00:00,2022-01-03 04:22:00+00:00,2022-01-03 18:48:00+00:00,2022-01-05 18:10:00+00:00,6.500000,2022,1,1,0
2,91079,73014,Processing,M,2022-01-01 00:15:00+00:00,NaT,NaT,NaT,1,132525,...,Processing,2021-12-31 20:56:48+00:00,NaT,NaT,NaT,6.500000,2022,1,1,0
3,87344,70024,Complete,F,2020-01-01 04:36:00+00:00,NaT,2020-01-03 07:24:00+00:00,2020-01-04 14:43:00+00:00,1,127115,...,Complete,2020-01-01 02:45:33+00:00,2020-01-03 07:24:00+00:00,2020-01-04 14:43:00+00:00,NaT,7.000000,2020,1,1,0
4,9429,7576,Shipped,F,2021-01-01 02:46:00+00:00,NaT,2021-01-01 23:54:00+00:00,NaT,1,13726,...,Shipped,2021-01-01 01:41:57+00:00,2021-01-01 23:54:00+00:00,NaT,NaT,8.000000,2021,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181051,105418,84622,Processing,F,2022-12-27 01:59:00+00:00,NaT,NaT,NaT,4,153078,...,Processing,2022-12-29 01:13:35+00:00,NaT,NaT,NaT,12.430000,2022,12,4,52
181052,51994,41724,Processing,F,2022-12-29 10:11:00+00:00,NaT,NaT,NaT,1,75703,...,Processing,2022-12-29 09:41:57+00:00,NaT,NaT,NaT,12.930000,2022,12,4,52
181053,26727,21442,Shipped,M,2021-12-31 02:47:00+00:00,NaT,2022-01-02 10:02:00+00:00,NaT,1,38929,...,Shipped,2021-12-31 02:32:53+00:00,2022-01-02 10:02:00+00:00,NaT,NaT,30.860001,2021,12,4,52
181054,103525,83020,Shipped,F,2022-12-29 09:26:00+00:00,NaT,2022-12-30 17:24:00+00:00,NaT,2,150331,...,Shipped,2022-12-29 05:38:42+00:00,2022-12-30 17:24:00+00:00,NaT,NaT,7.160000,2022,12,4,52


In [151]:
#Menyiapkan dataset untuk menghitung GMV selama 3 kuartal terakhir

df10['GMV'] = df10['sale_price']*df10['num_of_item']
df10

,order_id,user_id,status,gender,created_at,returned_at,shipped_at,delivered_at,num_of_item,id,...,created_at_1,shipped_at_1,delivered_at_1,returned_at_1,sale_price,year,month,quarter,week,GMV
0,84220,67533,Complete,F,2022-01-01 13:41:00+00:00,NaT,2022-01-04 10:56:00+00:00,2022-01-08 09:09:00+00:00,1,122658,...,2022-01-01 12:40:00+00:00,2022-01-04 10:56:00+00:00,2022-01-08 09:09:00+00:00,NaT,6.000000,2022,1,1,0,6.0
1,120689,96777,Returned,F,2022-01-01 07:04:00+00:00,2022-01-05 18:10:00+00:00,2022-01-03 04:22:00+00:00,2022-01-03 18:48:00+00:00,1,175242,...,2022-01-01 03:15:35+00:00,2022-01-03 04:22:00+00:00,2022-01-03 18:48:00+00:00,2022-01-05 18:10:00+00:00,6.500000,2022,1,1,0,6.5
2,91079,73014,Processing,M,2022-01-01 00:15:00+00:00,NaT,NaT,NaT,1,132525,...,2021-12-31 20:56:48+00:00,NaT,NaT,NaT,6.500000,2022,1,1,0,6.5
3,87344,70024,Complete,F,2020-01-01 04:36:00+00:00,NaT,2020-01-03 07:24:00+00:00,2020-01-04 14:43:00+00:00,1,127115,...,2020-01-01 02:45:33+00:00,2020-01-03 07:24:00+00:00,2020-01-04 14:43:00+00:00,NaT,7.000000,2020,1,1,0,7.0
4,9429,7576,Shipped,F,2021-01-01 02:46:00+00:00,NaT,2021-01-01 23:54:00+00:00,NaT,1,13726,...,2021-01-01 01:41:57+00:00,2021-01-01 23:54:00+00:00,NaT,NaT,8.000000,2021,1,1,0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181051,105418,84622,Processing,F,2022-12-27 01:59:00+00:00,NaT,NaT,NaT,4,153078,...,2022-12-29 01:13:35+00:00,NaT,NaT,NaT,12.430000,2022,12,4,52,49.720001
181052,51994,41724,Processing,F,2022-12-29 10:11:00+00:00,NaT,NaT,NaT,1,75703,...,2022-12-29 09:41:57+00:00,NaT,NaT,NaT,12.930000,2022,12,4,52,12.93
181053,26727,21442,Shipped,M,2021-12-31 02:47:00+00:00,NaT,2022-01-02 10:02:00+00:00,NaT,1,38929,...,2021-12-31 02:32:53+00:00,2022-01-02 10:02:00+00:00,NaT,NaT,30.860001,2021,12,4,52,30.860001
181054,103525,83020,Shipped,F,2022-12-29 09:26:00+00:00,NaT,2022-12-30 17:24:00+00:00,NaT,2,150331,...,2022-12-29 05:38:42+00:00,2022-12-30 17:24:00+00:00,NaT,NaT,7.160000,2022,12,4,52,14.32


In [152]:
#Sorting dataset untuk menghitung GMV selama 3 kuartal terakhir

df11 = df10.sort_values('quarter',ascending=False)

In [153]:
#Sorting dataset untuk menghitung GMV selama 3 kuartal terakhir

df12 = df11.groupby('quarter')[['GMV']].sum()
df12

,GMV
quarter,
1,6365017.846338
2,3560595.633881
3,4552110.335136
4,5942791.906657


Insight

Dari data GMV di atas terlihat bahwa angka pergerakan GMV selama periode Q1 Q3 2022 masih belum stabil

Kesimpulan Pertanyaan 3

Saya merasa bisnis The Look ini masih memiliki potensi kedepannya dikarenakan angka jumlah user & jumlah transaksi yang masih dalam berada trend kenaikan meskipun masih terdapat PR dalam menjaga trend kenaikan menjadi lebih stabil dari sisi jumlah user & total pendapatan. 

Dengan adanya dana tambahan dari investor baru, akan dinilai menjadi nilai tambah yang sangat berguna karena perusahaan bisa lebih menyiapkan campaign yang lebih menarik untuk meningkatkan customer retention & GMV di periode selanjutnya.